In [1]:
from custom_env import StockTradingEnv
import pandas as pd
import torch
import torch.nn as nn

class TrendPredictor(nn.Module):
    def __init__(self, window_size, hidden_size):
        super(TrendPredictor, self).__init__()
        self.window_size = window_size
        self.hidden_size = hidden_size
        self.LSTM = nn.LSTM(self.window_size, self.hidden_size, batch_first=True)
        self.do = nn.Dropout(0.1)
        self.fc = nn.Linear(self.hidden_size, 1)
    def forward(self, x):
        x, _ = self.LSTM(x)
        x = self.do(x)
        x = self.fc(x)
        return x

WINDOW_SIZE = 256
HIDDEN_SIZE = 512

trend_predictor = TrendPredictor(WINDOW_SIZE, HIDDEN_SIZE)
trend_predictor.load_state_dict(torch.load("LSTM_Trader.pt"))

<All keys matched successfully>

In [2]:
from stable_baselines3 import PPO
#(self, df, trend_predictor, predict_steps=5

df = pd.read_csv('dataset/stocks/A.csv')["Close"]
env = StockTradingEnv(df, trend_predictor, 10)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/herb/.local/lib/python3.13/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 41   |
|    iterations      | 1    |
|    time_elapsed    | 49   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 41           |
|    iterations           | 2            |
|    time_elapsed         | 99           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0013470526 |
|    clip_fraction        | 0.000439     |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.84        |
|    explained_variance   | 4.17e-06     |
|    learning_rate        | 0.0003       |
|    loss                 | 4.48e+05     |
|    n_updates            | 10           |
|    policy_gradient_loss | -7.01e-05    |
|    std                  | 1            |
|    value_loss           | 1.62e+06     |
----------------

In [3]:

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        break
env.render()

Balance: 0.00
Shares: 280.92
Portfolio Value: 19361.33


In [ ]:
# ну не круто разве) 19к с 10 чел поднял